In [ ]:
#!pip install nnfs 

In [83]:
import sys
import nnfs
import numpy as np
#np.random.seed(0)
nnfs.init()
from nnfs.datasets import spiral_data
import matplotlib.pyplot as plt
%matplotlib inline

### 4 Nueron with 1 output

In [17]:
inp=np.array([1,2,3,2.5])
weights=np.array([0.2,0.8,-0.5,1.])
bias=2

In [18]:
output=inp.dot(weights)+bias
output

4.8

### 1 input with 3 outputs

In [52]:
inp1=[1,2,3,2.5]

w=[
    [0.2,0.8,-0.5,1.],
    [0.5,-0.91,0.26,-0.5],
    [-0.26,-0.27,0.17,0.87]
]
b=[2,3,0.5]


In [60]:
output=np.dot(w,inp1)+b
print(output)

[4.8   1.21  2.385]


#### Batch inputs
**Note:** in batches we have to inp.dot(w.T)

In [67]:
i=[
    [1,2,3,2.5],
    [2.,5.,-1.,2.0],
    [-1.5,2.7,3.3,-0.8]
  
  ]
#1st layer
w=[
    [0.2,0.8,-0.5,1.],
    [0.5,-0.91,0.26,-0.5],
    [-0.26,-0.27,0.17,0.87]
   ]
b=[2,3,0.5]
#2nd layer
w2=[
    [0.1  , -0.14,  0.5],
    [-0.5 , 0.12 ,-0.33],
    [-0.44, 0.73 ,-0.13]
   ]
b2=[-1,2,-0.5]

In [68]:
layer1_output=np.dot(i,np.array(w).T)+b
print(layer1_output)

[[ 4.8    1.21   2.385]
 [ 8.9   -1.81   0.2  ]
 [ 1.41   1.051  0.026]]


In [70]:
layer2_output=np.dot(layer1_output,np.array(w2).T)+b2
print(layer2_output)

[[ 0.5031  -1.04185 -2.03875]
 [ 0.2434  -2.7332  -5.7633 ]
 [-0.99314  1.41254 -0.35655]]


In [74]:
class Layer_Dense:
    def __init__(self,n_inputs,n_nueron):
        self.weights = 0.1 * np.random.randn(n_inputs,n_nueron)
        self.biases  = np.zeros((1,n_nueron))
    def forward(self,inputs):
        self.output=np.dot(inputs,self.weights)+self.biases
        

In [85]:
class Activation_ReLU:
    def forward(self,inputs):
        self.output=np.maximum(0,inputs)

In [97]:
class Activation_SoftMax:
    def forward(self,inputs):
        exp_val=np.exp(inputs- np.max(inputs,axis=1,keepdims=True))
        self.output=exp_val/np.sum(exp_val,axis=1,keepdims=True)

In [110]:
class Loss:
    def Calculate(self,desire,actual):
        sample_losses = self.forward(desire,actual)
        data_loss = np.mean(sample_losses)
        return data_loss

In [111]:
class Loss_CategoricalCrossEntropy(Loss):
    def forward(self,y_pred,y_true):
        samples = len(y_pred)
        y_pred_clipped = np.clip(y_pred,1e-7,1-1e-7)
        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[range(samples),y_true]
        elif  len(y_true.shape) == 2:
            correct_confidences = np.sum(y_pred_clipped*y_true,axis=1)
        neg_log_likelihoods = -np.log(correct_confidences)   
        return neg_log_likelihoods

In [75]:
X=[
    [1,2,3,2.5],
    [2.,5.,-1.,2.0],
    [-1.5,2.7,3.3,-0.8]
  
  ]

In [76]:
layer_1=Layer_Dense(4,5)
layer_2=Layer_Dense(5,2)

In [ ]:
layer_1.forward(X)
print(layer_1.output)

In [ ]:
layer_2.forward(layer_1.output)
print(layer_2.output)

In [84]:
X,y = spiral_data(100,3)

### Activation ReLU 

In [ ]:
layer1=Layer_Dense(2,5)
act1=Activation_ReLU()
layer1.forward(X)
act1.forward(layer1.output)
print(act1.output)

### Activation Softmax

In [94]:
inputs=[
    [4.8,1.21,2.385],
    [8.9,-1.81,0.2],
    [1.41,1.051,0.026]
]

In [98]:
softmax=Activation_SoftMax()
softmax.forward(inputs)
print(softmax.output)

[[8.95282664e-01 2.47083068e-02 8.00090293e-02]
 [9.99811129e-01 2.23163963e-05 1.66554348e-04]
 [5.13097164e-01 3.58333899e-01 1.28568936e-01]]


In [99]:
X,y = spiral_data(100,3)

In [102]:
dense1=Layer_Dense(2,3)
act1=Activation_ReLU()

dense2=Layer_Dense(3,3)
act2=Activation_SoftMax()

dense1.forward(X)
act1.forward(dense1.output)

dense2.forward(act1.output)
act2.forward(dense2.output)

print(act2.output[:5])

[[0.33333334 0.33333334 0.33333334]
 [0.3333159  0.33334544 0.33333868]
 [0.333321   0.33334187 0.3333371 ]
 [0.33330086 0.33335584 0.33334327]
 [0.33327627 0.33337292 0.3333508 ]]


### Categorical Cross-Entropy
$-\sum_{{i,j}=1}^{n} d_i*log(y_j)$<br>
**where** <br>$d_i$ is desire output <br>
$y_j$ is actual/predicted output

In [109]:
import math
# let its an softmax output
softmax_output = [0.7,0.1,0.2]
#one hot encoding
target_output = [1,0,0]
loss=-(target_output[0] * math.log(softmax_output[0])
      + target_output[1] * math.log(softmax_output[1])
      + target_output[2]  * math.log(softmax_output[2])
      )
print(loss)

0.35667494393873245


In [112]:
loss_function=Loss_CategoricalCrossEntropy()
loss=loss_function.Calculate(act2.output,y)
print(loss)

1.0983953


In [1]:
import math
import random
import numpy as np
np.random.seed(0)
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
def init_weights_biases():
    # weights and bias should be -0.5 to 0.5
    # 28,28 pixels will 784 vector and hidden layer has 10 nueron so (10,784)
    W1 = np.random.rand(10, 784) - 0.5  
    # every nueron has unique bias
    b1 = np.random.rand(10, 1) - 0.5
    # output layer will have 10 ---> 0-9 = 10 so from hidden nueron 10 will 
    W2 = np.random.rand(10, 10) - 0.5
    # every nueron has unique bias
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

In [3]:
def Activation_ReLU(i_out):
    return np.maximum(i_out, 0) # 0 for -inf to 0 for others will be i_out itself

In [4]:
def Activation_ReLU_deriv(i_out):
    return i_out > 0 # due to ReLU function curve either 0 , 1 = False , True

In [5]:
def Activation_Softmax(j_out):
    act = np.exp(j_out) / sum(np.exp(j_out))
    return act

In [6]:
def Activation_Sigmoid(j_out):
    exp_val=np.exp(-j_out)
    return 1/(1+exp_val)

In [7]:
def calculate_outputs(W1, b1, W2, b2, X):
    #(10,784).(784,rows)=(10,rows)
    i_out = W1.dot(X) + b1
    reLU = Activation_ReLU(i_out)
    #(1,10).(10,rows)=(1,rows)
    j_out = W2.dot(reLU) + b2
    s_max = Activation_Softmax(j_out)
    return i_out, reLU, j_out, s_max

In [8]:
def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

In [9]:
def calculate_error(i_out, reLU, j_out, s_max, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    #(1,rows)
    dj_out = s_max - one_hot_Y
    #(1,rows).(rows,10)
    dW2 = 1 / rows * dj_out.dot(reLU.T)
    db2 = 1 / rows * np.sum(dj_out)
    #(10,1).(1,rows) = (10,rows)
    di_out = W2.T.dot(dj_out) * Activation_ReLU_deriv(i_out)
    #(10,rows).(rows,784)
    dW1 = 1 / rows * di_out.dot(X.T)
    db1 = 1 / rows * np.sum(di_out)
    return dW1, db1, dW2, db2

In [10]:
def update_wbs(W1, b1, W2, b2, dW1, db1, dW2, db2, rate):
    W1 = W1 - rate * dW1
    b1 = b1 - rate * db1    
    W2 = W2 - rate * dW2  
    b2 = b2 - rate * db2    
    return W1, b1, W2, b2

In [11]:
def predict(s_max):
    return np.argmax(s_max, 0) # will return maximum value index

In [12]:
def cal_accuracy(y_cap, Y):
    return (np.sum(y_cap == Y) / Y.size)*100 # how many times pred and y are equals

In [13]:
def gradient_descent(X, Y, rate, epochs):
    W1, b1, W2, b2 = init_weights_biases()
    for i in range(epochs):
        i_out, reLU, j_out, s_max = calculate_outputs(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = calculate_error(i_out, reLU, j_out, s_max, W1, W2, X, Y)
        W1, b1, W2, b2 = update_wbs(W1, b1, W2, b2, dW1, db1, dW2, db2, rate)
        if i % 100 == 0:
            print("Iteration: ", i)
            y_cap = predict(s_max)
        
            print("Accuracy:",cal_accuracy(y_cap, Y))
    return W1, b1, W2, b2

In [14]:
def cal_predictions(X, W1, b1, W2, b2):
    i_out, reLU, j_out, s_max = calculate_outputs(W1, b1, W2, b2, X)
    y_cap = predict(s_max)
    return y_cap

In [15]:
def check_prediction(image, label,W1, b1, W2, b2):
    
    y_cap = cal_predictions(image, W1, b1, W2, b2)
    
    print("Prediction: ",  y_cap)
    print("Label: ", label)
    
    image = image.reshape((28, 28)) * 255
    plt.imshow(image)
    plt.show()

In [ ]:
data = np.array(ds.values)
data = data.T #(785,m)
Y = data [0]
X = data [1:]
X = X / 255.
cols,rows = X.shape
cols,rows, Y.shape

In [ ]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.5, 501)

In [ ]:
for i in range(10):
    image=np.array(test.T.iloc[:,i])
    label=image[0]
    image=image[1:,None]/255
    check_prediction(image,label, W1, b1, W2, b2)